De forma geral será feito um agente que possui observações parciais do ambiente, tendo como objetivo encontrar um caminho até o objetivo.

Como característica do ambiente temos que ele é determinístico, desconhecido, parcialmente observável, suas ações serão dependentes (time-series), estático, suas percepções serão determinísticas e haverá apenas um agente no ambiente (single agent).

Com base nas características do ambiente será construído um agente, sendo um agente baseado em objetivos com representação atômica, ou seja, não haverá estrutura na qual o agente poderá inferir informações, sua única capacidade será, dado um estado, dizer se é o objetivo ou não, sendo esse tipo de agente é chamado busca, (ideal é que esse ambiente fosse totalmente conhecido, depois o tornaremos)

In [1]:
import numpy as np

In [2]:
"""Iremos criar a classe onde a parte abstrata da busca estará ocorrendo.
Segundo o livro essa estrutura pode ser representada como uma árvore e que em cada nó deve conter o estado do nó 'estado' 
(Como complemento ao estado adicionamos outro atributo 'local'), 
ação que formou o nó 'acao', o pai do nó atual 'parent', e o 'custo' da raiz até o nó atual.
E por fim além dos atributos precisamos de um método para avançar para os próximos estados 'expandir', no livro está descrito como 
'modelo de transição'"""

class node:

    def __init__(self, estado, local, acao=None, parent=None, custo=0, passo_redundante=[]):
        self.estado = estado
        self.acao = acao
        self.custo = custo
        self.local = local
        self.parent = parent
        self.passo_redundante = passo_redundante
        
    def __repr__(self):
        return "<Node {}>".format(self.estado)
         
# Método utilizado para expandir (avançar) para os próximos estados
    def expandir(self, problema):

        return [self.no_filho(problema, acoes)
                for acoes in problema.acoes_pos(self.estado, self.local, self.passo_redundante)]
    
# Esse método será utilizado pelo 'expandir' para realmente demostrar o novo estado
    def no_filho(self, problema, acao):

        proximo_estado = problema.resultado(self.estado, acao, self.local)
        
        if acao == 'cima':
            local = [self.local[0]-1, self.local[1]]
        elif acao == 'baixo':
            local = [self.local[0]+1,self.local[1]]
        elif acao == 'esquerda':
            local = [self.local[0],self.local[1]-1]
        else:
            local = [self.local[0],self.local[1]+1]

        prox_node = node(proximo_estado, local, acao, self, problema.custo_caminho(self.custo))

        return prox_node
    
    """Devido aos parâmetros inicializados com a classe podemos refazer os passos, então depois de 
    encontrado o objetivo, será feito um caminho inverso demostrando as ações tomadas"""
    def solucao(self):

        return [no.acao for no in self.cam()[::-1][1:]]
    
# Sendo cam o método que irá auxiliar a 'solucao' a fazer o caminho inverso
    def cam(self):

        nodee, ca = self, []
        while nodee:
            ca.append(nodee)
            nodee = nodee.parent
        return(list(ca))

In [3]:
"""Aqui irá acontecer a definição em si do problema, suas regras, possíveis ações, resultado das ações, teste de objetivo, etc.
os métodos e sua construção irá depender muito do problema à ser abordado"""

class caminho:

    def __init__(self, inicial, objetivo):
        self.inicial = inicial
        self.objetivo = objetivo
        dim = inicial.shape
        self.dim = [dim[0]-1, dim[1]-1]

    def custo_caminho(self, c):

        return c+1

    def acoes_pos(self, estado, local, passo_redundante):

        acoes = []

        """Variáveis utilizadas para localização caso o agente esteja cercado por 
        estados já. Permitindo passos redundantes"""
        cnt_sup_dir = 0
        cnt_sup_esq = 0
        cnt_inf_dir = 0
        cnt_inf_esq = 0
        row_sup = 0
        row_inf = 0
        col_esq = 0
        col_dir = 0

        # esta na primeira coluna
        if local[1] == 0:
            col_esq+=1

            # Canto superior esquerdo
            if local == [0,0]:
                cnt_sup_esq+=1

                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:
    
                    acoes.append('baixo')

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       
         
                    acoes.append('direita')

            # Canto inferior esquerdo
            elif local == [self.dim[0],0]:
                cnt_inf_esq+=1
                        
                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  

                    acoes.append('cima') 

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       
    
                    acoes.append('direita')

            else:
                col_esq+=1
                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:
            
                    acoes.append('baixo')

                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  
               
                    acoes.append('cima')

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       
          
                    acoes.append('direita')               
        
        # esta na primeira fileira
        elif local[0] == 0:
            row_sup+=1
                    
            # Canto superior direito
            if local == [0,self.dim[0]]:
                cnt_sup_dir+=1

                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:

                        acoes.append('baixo')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:
                        
                        acoes.append('esquerda')

            # Canto superior esquerdo
            elif local == [0,0]:
                cnt_sup_esq+=1

                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:

                    acoes.append('baixo')

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       

                    acoes.append('direita')

            else:
           
                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:

                    acoes.append('baixo')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:

                    acoes.append('esquerda')

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       

                    acoes.append('direita')

        # esta na ultima fileira
        elif local[0] == self.dim[0]:
            row_inf+=1

            # Canto inferior direito
            if local == [self.dim[0],self.dim[1]]:
                cnt_inf_dir+=1

                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  

                    acoes.append('cima')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:

                    acoes.append('esquerda')

            # Canto inferior esquerdo
            elif local == [self.dim[0],0]:
                cnt_inf_esq+=1

                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  

                    acoes.append('cima') 

                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       
 
                    acoes.append('direita')

            else:

                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  

                    acoes.append('cima')
                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:
                  
                    acoes.append('esquerda')
                # testar se veio da direita
                if estado[local[0]][local[1]+1] != 1:       
                  
                    acoes.append('direita')

        # esta na ultima coluna
        elif local[1] == self.dim[1]:
            col_dir+=1
                                 
            # Canto inferior direito
            if local == [self.dim[0],self.dim[1]]:
                cnt_inf_dir+=1
     
                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  
              
                    acoes.append('cima')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:
                  
                    acoes.append('esquerda')

            # Canto superior direito
            elif local == [0,self.dim[1]]:
                cnt_sup_dir+=1

                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:
                   
                        acoes.append('baixo')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:
                   
                        acoes.append('esquerda')

            else:

                # testar se veio de baixo
                if estado[local[0]+1][local[1]] != 1:
            
                    acoes.append('baixo')

                # testar se veio de cima
                if estado[local[0]-1][local[1]] != 1:  
          
                    acoes.append('cima')

                # testar se veio da esquerda
                if estado[local[0]][local[1]-1] != 1:

                    acoes.append('esquerda')

        else:
                   
            # testar se veio de baixo
            if estado[local[0]+1][local[1]] != 1:

                acoes.append('baixo')
            # testar se veio de cima
            if estado[local[0]-1][local[1]] != 1:  

                acoes.append('cima') 
            # testar se veio da esquerda
            if estado[local[0]][local[1]-1] != 1:

                acoes.append('esquerda')
            # testar se veio da direita
            if estado[local[0]][local[1]+1] != 1:
                
                acoes.append('direita')

        """Caso o agente esteja em um local cercado por 1, ou seja, que já tenham sido visitados
        e pode haver parede. Ele será permitido a fazer um passo redundante, avançando para um local já
        visitado, isso será possível com as variáveis iniciadas no começo do método"""

        if len(acoes) == 0:
            # Iremos adicionar 1 há uma lista, dessa forma iremos saber a quantia de passos redundantes
            passo_redundante.append(1)
          
            if row_sup >= 1:
                
                if cnt_sup_dir >= 1:
                    acoes = ['baixo', 'esquerda']
                elif cnt_sup_esq >= 1:
                    acoes = ['baixo', 'direita']
                else:
                    acoes = ['baixo', 'esquerda', 'direita']

            elif row_inf >= 1:

                if cnt_inf_dir >= 1:
                    acoes = ['cima', 'esquerda']

                elif cnt_inf_esq >= 1:
                    acoes = ['cima', 'direita']

                else:
                    acoes = ['cima', 'esquerda', 'direita']

            elif col_dir >= 1:

                if cnt_inf_dir >= 1:
                    acoes = ['cima', 'esquerda']

                elif cnt_sup_dir >= 1:
                    acoes = ['baixo', 'esquerda']

                else:
                    acoes = ['cima', 'baixo', 'esquerda']

            elif col_esq >= 1:

                if cnt_inf_esq >= 1:
                    acoes = ['cima', 'direita']

                elif cnt_sup_esq >= 1:
                    acoes = ['baixo', 'direita']

                else:
                    acoes = ['cima', 'baixo', 'direita']
                
            else:
                acoes = ['cima', 'baixo', 'esquerda', 'direita']

        return acoes

    # Dado uma ação ira demostrar seu impacto no estado
    def resultado(self, estado, acao, local):

        try:
            estado = estado.copy()
        except:
            estado = estado.estado.copy()
            local = estado.local

        if acao == 'cima':
            estado[local[0]-1][local[1]] = 1

        elif acao == 'baixo':
            estado[local[0]+1][local[1]] = 1

        elif acao == 'esquerda':
            estado[local[0]][local[1]-1] = 1

        elif acao == 'direita':
            estado[local[0]][local[1]+1] = 1

        return estado

    # E por fim será testado se determinado estado é o objetivo
    def teste_objetivo(self, estado):

        if estado[self.objetivo[0]][self.objetivo[1]] == 1:
             return True
        else:
             return False

E por fim, existem alguns algoritmos que podem ser usados para busca em árvores ou grafos. Uma das distinções de um algoritmo para o outro é a forma com a qual ele irá expandir os nós, isso irá afetar em sua performasse, essa pode ser mensurada por 4 métricas.
- Velocidade em que o algoritmo encontrou o objetivo (complexidade de tempo)
- A quantia de espaço que é utilizado (complexidade de espaço)
- Se caso o objetivo pode ser encontrado o algoritmo fará isso com certeza (completude)
- Se esse objetivo encontrado é pelo caminho ótimo (Otimização)

Dessa forma cada algoritmo terá seu ponto forte, sendo mais recomendado dependendo do tipo do problema, e para que está sendo utilizado

In [4]:
ambiente = np.zeros([10,10])

# Local de início do agente
ambiente[7,6] = 1
problema = caminho(ambiente, objetivo=[3,4])

In [5]:
"""Esse algoritmo irá expandir o ramo da estrutura até o seu fim, caso chegue e não tenha encontrado o objetivo
será continuado em outro ramo até que o objetivo seja encontrado(supondo que ele pode ser encontrado).
É um algoritmo completo
Não é ótimo (seu caminho encontrado pode não ser o melhor)

"""

def Profundidade(prob, local):

    # Lista com os nós expandidos
    folha = [node(prob.inicial, local)] 

    if prob.teste_objetivo(folha[0].estado):
        return folha[0]
    
    while folha:

        # Irá selecionar o último nó
        no = folha.pop()

        if prob.teste_objetivo(no.estado):
            return no
        
        # Irá expandir a lista de folhas, adicionando as novas no fim
        folha.extend(no.expandir(prob))

    return None

In [6]:
busca_profundidade = Profundidade(problema, [7,6])

print(busca_profundidade.estado)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [7]:
"""Esse algoritmo irá expandir sempre o nó mais curto, ou seja, ele fará seu caminho 
da esquerda para direita para se caso não haver mais nós no nível ir para o próximo

É um algoritmo completo
Caso o custo de caminho for uniforme (Como é o caso) ele será ótimo


"""

def Largura(prob, local):

    # Lista com os nós
    folha = [node(prob.inicial, local)]
    
    if prob.teste_objetivo(folha[0].estado):
        return folha[0]

    while folha:

        # Uma lista com os nós recém expandidos
        test = folha[0].expandir(prob)
       
        """Há um detalhe no algoritmo que ele fará o teste nos nós recém expandidos
        ao invés de fazer isso na hora que os nós forem expandidos. Isso tem um motivo,
        fazendo isso reduzimos um nível na árvore (sendo esse ultimo nível extremamente custoso)"""
        for i in range(len(test)):

            if prob.teste_objetivo(test[i].estado):
                return test[i]
            
            folha.extend([test[i]])
        # É deletado o primeiro nó da lista desta forma 'test' pegará sempre o "vizinho"    
        folha.remove(folha[0])
      

In [8]:
busca_largura = Largura(problema, [7,6])

print(busca_largura.estado)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Algoritmo Uniform-Cost Search como nome já diz ele irá buscar o caminho ótimo, diferente do algoritmo em largura
essa condição não depende do custo ser constatante. Aqui estão os passos de como isso é feito:
Será mantido uma lista como 'folhas' e 'explorados'(contendo os nós que já foram testado o  objetivo)
- Em 'folha' conterá todos os nós que não foram feito o teste objetivo
- Selecionar o nó em 'folha' com menor custo e fazer o teste de objetivo
- Caso o nó não seja o objetivo (se for encontramos a solução) ele será expandido, estará na lista filhos
- Em filhos será feito o teste se já estão no 'explorados' caso não, será testado  se estão no 
'folha' (Falso:serão adicionados) caso estejam será feito o teste se o nó em filho tem menor custo que o que está em folha,
caso for verdadeiro haverá a troca.
- O nó que acabamos de testar objetivo será excluído de 'folha' e adicionado em 'explorados'

Ele conquista o caminho ótimo: 1 selecionando o nó com menor custo e 2 (diferente do algo em largura) O teste de objetivo
só acontece quando o nó for expandido (caso contrario não tenhamos o caminho ótimo, mesmo cumprindo 1)

Perceba também que no ambiente em que o algoritmo está sendo utilizado os últimos 2 pontos comentados se tornam desnecessários
já que a forma em que o ambiente foi definido não ocorre estados repetidos, logo não precisamos testar se o nó já esta em 'expandido'
ou 'folha'

In [9]:
def Custo_uniforme(prob, local):

    folha = [node(prob.inicial, local)]

    visitado = []

    if prob.teste_objetivo(folha[0].estado):
        return folha
    
    folha.extend(folha[0].expandir(prob))
    visitado.append(folha[0])

    while folha:

        folha_min = folha[0]

        # Selecionar o nó em folha com menor custo
        for i in range(len(folha)):

            if folha_min.custo > folha[i].custo:
                folha_min = folha[i]

        # Testar se o nó com menor custo é o objetivo
        if prob.teste_objetivo(folha_min.estado):
            return folha_min
        
        filhos = folha_min.expandir(prob)

        # Fazer os testes dos filhos
        for filho in filhos:
            if filho not in visitado: 
                if filho not in folha:
                    folha.append(filho)
                else:
                    # comparar se o estado em folha tem custo maior ou menor que o novo no
                    for i in folha:
                        if i == filho:
                            if i.custo > filho.custo:
                                folha.remove(i)
                                folha.append(filho)
                    
        folha.remove(folha_min)
        visitado.append(folha_min)

In [10]:
busca_uniforme = Custo_uniforme(problema, [7,6])

print(busca_uniforme.estado)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Depth-limited search é uma busca em profundidade com uma pequena diferença, ele irá até 'limite' ou seja serão expandidos uma certa quantia de nós em uma direção, caso não encontre nesse determinado numero, irá tentar outros caminhos. Sua utilidade ocorre quando a árvore não possui o objetivo no ramo em que está sendo expandido e há infinitos nós por aquele ramo (isso pode acontecer por nós repetidos, os chamados laços), claramente a busca em profundidade ficaria preso em tal ambiente, coisa que a busca limitada iria poder lidar

In [11]:
def profundidade_lim(prob, local, limite=30):

    def profundidade_recursiva(no, prob, limite):
    
        if prob.teste_objetivo(no.estado):  
            return no
        elif limite == 0:
            return 'fim'
        else:
            terminar = False
            for filho in no.expandir(prob):

                resultado = profundidade_recursiva(filho, prob, limite - 1)
                if resultado == 'fim':
                    terminar = True
                elif resultado is not None:
                    return resultado

            return 'fim' if terminar else None
        
    return profundidade_recursiva(node(prob.inicial, local), prob, limite)

In [12]:
busca_profundidade_lim = profundidade_lim(problema, [7,6])

print(busca_profundidade_lim.estado)

[[0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]]


In [13]:
"""E perceba que caso saibamos a quantia de passos até o objetivo podemos 
encontra-lo no de forma ótima caso os passos forem uniforme, mesma condição 
que a busca em largura (claro que tal informação é bem improvável)"""
busca_profundidade_lim = profundidade_lim(problema, [7,6], 6)

print(busca_profundidade_lim.estado)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Como foi dito anteriormente, o algoritmo de profundidade limitada pode trazer no mínimo o caminho ótimo sob determinadas condições e irá retornar no mínimo um caminho melhor (mais curto, menos custoso) que algoritmos em largura ou largura limitada (caso esse limite não seja o ótimo). Porem há uma solução, podemos incrementar esse limite de um em um, quando encontrar a solução irá retornar o caminho, sendo esse com certeza o limite, profundidade ótima.
Outra vantagem dessa abordagem é em questão à memória que passa a ser usada menos.

In [14]:
"""O 'problema' desse algoritmo é que os níveis serão buscados varias vezes, ou seja se o limite n for 
o ótimo, limite 1 será buscado n vezes, nível (n-1) 2 vezes e assim por diante, porem a complexidade 
asymptotic é a mesma tando para o iterativo quanto para o limitado"""

# o 'max_profundidade' é o nível máximo alcançado, até onde a iteração vai
def profundidade_iterativa(prob, local, max_profundidade):
 
    for limite in range(max_profundidade):

        resultado = profundidade_lim(prob, local, limite)
       
        if resultado != 'fim':
            return resultado

In [15]:
busca_profundidade_iter = profundidade_iterativa(problema, [7,6], 200)

print(busca_profundidade_iter.estado)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Uma das preocupações é a questão de várias repetições de níveis feita pelo algoritmo, em alguns casos em que o ambiente demande um uso muito grande de memória outros algoritmos com o profundidade limitada se tornam impossíveis, havendo então a solução de manter os níveis até onde a memória suportar armazenados e passar a usar a iteração a partir desse nível 

Até agora viemos falando de uma busca onde não era possível ter a observação completa, faltava saber onde o objetivo se encontrava, em alguns casos temos essa possibilidade sendo então usados algoritmos específicos, demostrados agora.

In [16]:
# Definindo heurística 
def manhattan(local, objetivo):

    return ((local[0] - objetivo[0])**2 + (local[1] - objetivo[1])**2)**0.5

In [17]:
def greedy_search(prob, local, h):

    # Lista contendo os nós ainda não testados
    folha = [node(prob.inicial, local)]

    # Testar se nó raiz é o objetivo
    if prob.teste_objetivo(folha[0].estado):
        return folha[0]

    while folha:
        # O interesse é selecionar qualquer nó 
        no = folha[0]

        # Selecionar o nó da lista folha com melhor heurística 
        for i in range(1, len(folha)):
            if h(no.local, prob.objetivo) > h(folha[i].local, prob.objetivo):
                no = folha[i]

        # Depois do nó selecionado testar se é o objetivo
        if prob.teste_objetivo(no.estado):
            return no
        
        # Estender o nó testado os colocando em 'folha'
        folha.extend(no.expandir(prob))
        # Remover da lista o nó recém testado
        folha.remove(no)

In [18]:
busca_greedy = greedy_search(problema, [7,6], manhattan)

print(busca_greedy.estado)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [19]:
"""a-star é também um algoritmo que estará utilizando uma heurística, mas diferente do greedy seu função f 
(que irá decidir o nó escolhido para para testar o objetivo) é feito pela soma de h(manhattan) e o custo 
para chegar da raiz até esse nó. Essa pequena mudança faz total diferença, se esse h for consistente e 
admissível temos que a-star será ótima e completa"""

def a_star(prob, local, h):

    # Lista contendo os nós ainda não testados
    folha = [node(prob.inicial, local)]

    # Testar se nó raiz é o objetivo
    if prob.teste_objetivo(folha[0].estado):
        return folha[0]

    while folha:
        # O interesse é selecionar qualquer nó 
        no = folha[0]

        # Selecionar o nó da lista folha com melhor heurística 
        for i in range(1, len(folha)):
            if h(no.local, prob.objetivo)+no.custo > h(folha[i].local, prob.objetivo)+folha[i].custo:
                no = folha[i]

        # Depois do nó selecionado testar se é o objetivo
        if prob.teste_objetivo(no.estado):
            return no
        
        # Estender o nó testado os colocando em 'folha'
        folha.extend(no.expandir(prob))
        # Remover da lista o nó recém testado
        folha.remove(no)

In [20]:
busca_a_star = a_star(problema, [7,6], manhattan)

print(busca_a_star.estado)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Apesar de A* ser um excelente algoritmo ainda consome memória exponencialmente, tornando difícil a solução para problemas maiores ou com uma taxa de expansão muito grande. 
A taxa de expansão do atual ambiente é 4, resultando então caso o objetivo esteja no nível d da árvore um armazenamento de 4^d nós, (não é uma análise precisa, é uma estimativa do máximo, até porque, a taxa de expansão a partir do primeiro passo é de no máximo 3)

In [25]:
"""Esse algoritmo irá tentar fazer o mesmo que a busca em profundidade limitada, sendo 
o próximo limite, o nó com o menor custo f porem que já excedeu o limite atual, é 
trabalhado também com uma volta dos valores f de menor custo para seu pai, dessa forma
é evitado expandir e adicionar novos caminhos (Isso tem um custo alto em questão a tempo
ainda mais nesse ambiente que o custo é uniforme).
O ponto forte do algoritmo é poder podar algumas ramificações com base em seu custo
assim teremos menos estados armazenados, o problema é que nesse ambiente o custo é uniforme
não ocorrendo então essa poda"""

def a_star_recursivo(prob, local, h):

    def RBFS(prob, no, flimite):
        if prob.teste_objetivo(no.estado):
            return no, 0  
        sucessor = no.expandir(prob)
        if len(sucessor) == 0:
            return None, np.inf
        for s in sucessor:
         
            s.f = max(s.custo + h(s.local,prob.objetivo), no.f)
        while True:

            sucessor.sort(key=lambda x: x.f)
            melhor = sucessor[0]

            if melhor.f > flimite:
                return None, melhor.f
            if len(sucessor) > 1:
                segund_melhor = sucessor[1].f
            else:
                segund_melhor = np.inf
            result, melhor.f = RBFS(prob, melhor, min(flimite, segund_melhor))
            if result is not None:
                return result, melhor.f

    folha = node(prob.inicial, local)
    folha.f = h(folha.local, prob.objetivo)
    resultado, melhorf = RBFS(prob, folha, np.inf)
    return resultado

In [26]:
busca_astar_recursivo = a_star_recursivo(problema, [7,6], manhattan)

print(busca_astar_recursivo.estado)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
def Busca_bidirecional(prob, local, local_objetivo):

    # Lista com estados 
    folha_ida = [node(prob.inital, local)]
    folha_volta = [node(prob.inital, local_objetivo)]

    if folha_ida[0].estado == folha_volta[0]:
        return folha_ida[0]
    
    # Loop para encontrar o caminho
    while folha_ida and folha_volta:


        filhos_ida = folha_ida[0].expandir(prob)

        for filho in filhos_ida:

            if filho in folha_volta:
                # Caminho encontrado
        
        folha_ida.remove(folha_ida[0])
        folha_ida.extend(filhos_ida)

        filhos_volta = folha_volta[0].expandir(prob)

        for filho in filhos_volta:

            if filho in folha_volta:
                # Caminho encontrado

        folha_volta.remove(folha_volta[0])
        folha_volta.extend(filhos_volta)

Áte agora as buscas estavam sendo feitas em espaços bem limitados, o problema fica mais sério se começarmos a coloca-lo em uma matriz 1000x1000, nesse tipo de local em situações que o caminho ótimo é mais de 500 passos a computação não é possível, (analisar o tamanho dos estados armazenados no caso A* não é fácil, há algumas variáveis a ser levadas em consideração) se tornando um problema armazenar tantos estados. Uma solução seria utilizar algoritmos que possui uma capacidade maior de armazenamento (em troca possivelmente de menos velocidade), o problema que os que tenho conhecimento desenvolvem um menores listas de estados com base no custo de passo, (mantendo apenas os mais promissores), porem nesse caso eles não  iram funcionar muito bem por conta do custo ser constante, não havendo o aproveitamento de seus  pontos fortes.